<a href="https://colab.research.google.com/github/Vladimirsp81/DLS_2/blob/master/HW_TextSummarization_Vladimir_Prudnikov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization. Homework

Всем привет! Это домашка по суммаризации текста.

На семинаре мы рассмотрели базовые модели для суммаризации текста. Попробуйте теперь улучшить два метода: TextRank и Extractive RNN. Задание достаточно большое и требует хорошую фантазию, тут можно эксперементировать во всю.

Для сдачи заданий надо получить определенное качество по test-у:

- 1 задание: 0.27 BLEU
- 2 задание: 0.3 BLEU

Если ваш подход пробивает это качество – задание считается пройденным. Плюсом будет описание того, почему вы решили использовать то или иное решение. 

Датасет: gazeta.ru

**P.S.** Возможно, в датасете находятся пустые данные. Проверьте эту гипотезу, и если надо, сделайте предобратоку датасета.


`Ноутбук создан на основе семинара Гусева Ильи на кафедре компьютерной лингвистики МФТИ.`

Загрузим датасет и необходимые библиотеки

In [0]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [0]:
!pip install --upgrade razdel allennlp torch fasttext OpenNMT-py networkx pymorphy2 nltk rouge==0.3.1 summa
!pip install transformers youtokentome catalyst

     |████████████████████████████████| 7.6MB 3.9MB/s 
Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.5.0+cu101)
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 194kB 40.8MB/s 
Requirement already up-to-date: networkx in /usr/local/lib/python3.6/dist-packages (2.4)
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 1.4MB 41.6MB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 163kB 40.6MB/s 
     |████████████████████████████████| 266kB 46.4MB/s 
     |████████████████████████████████| 30.9MB 100kB/s 
     |████████████████████████████████| 204kB 41.3MB/s 
     |████████████████████████████████| 133kB 46.8MB/s 
     |████████████████████████████████| 245kB 42.0MB/s 
     |████████████████████████████████| 5.2MB 33.9MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 51

In [0]:
import random
import pandas as pd

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(eval(line)) # Simple hack
    records = pd.DataFrame(records)
    if sort_by_date:
        records = records.sort("date")
    if shuffle:
        records = records.sample(frac=1)
    return records

In [0]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

## 1 задание: TextRank (порог: 0.27 BLEU)

TextRank - unsupervised метод для составления кратких выжимок из текста. 
Описание метода:

1. Сплитим текст по предложениям
2. Считаем "похожесть" предложений между собой
3. Строим граф предложений с взвешенными ребрами
4. С помощью алгоритм PageRank получаем наиболее важные предложения, на основе которых делаем summary.

Функция похожести можно сделать и из нейросетевых(или около) моделек: FastText, ELMO и BERT. Выберете один метод, загрузите предобученную модель и с ее помощью для каждого предложениия сделайте sentence embedding. С помощью косинусной меры определяйте похожесть предложений.

Предобученные модели можно взять по [ссылке](http://docs.deeppavlov.ai/en/master/features/pretrained_vectors.html).

In [0]:
!wget http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz

--2020-05-26 20:06:56--  http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662024852 (631M) [application/octet-stream]
Saving to: ‘rubert_cased_L-12_H-768_A-12_pt.tar.gz’

rubert_cased_L-12_H 100%[===================>] 631.36M  8.69MB/s    in 92s     

2020-05-26 20:08:29 (6.85 MB/s) - ‘rubert_cased_L-12_H-768_A-12_pt.tar.gz’ saved [662024852/662024852]



In [0]:
!tar -xf '/content/rubert_cased_L-12_H-768_A-12_pt.tar.gz'

In [0]:
!pip install deeppavlov

     |████████████████████████████████| 778kB 5.7MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 8.0MB 14.2MB/s 
     |████████████████████████████████| 1.5MB 56.5MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 7.3MB 40.3MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 665kB 49.9MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 2.1MB 48.1MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 20.1MB 7.8MB/s 
     |████████████████████████████████| 10.4MB 40.7MB/s 
     |████████████████████████████████| 6.7MB 8.8MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 2.7MB 46.2MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |███

In [0]:
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs

In [0]:
bert_config = read_json(configs.embedder.bert_embedder)
bert_config['metadata']['variables']['BERT_PATH'] = '/content/rubert_cased_L-12_H-768_A-12_pt'

model = build_model(bert_config)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


In [0]:
import scipy
import razdel

In [0]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [0]:
%%time
from itertools import combinations
import networkx as nx
import numpy as np
import pymorphy2

# Используйте эту штуку как бэйзлайн
def unique_words_similarity(words1, words2):
    '''
    Функция подсчёта близости предложений на основе пересечения слов
    ''' 
    words1 = set(words1)
    words2 = set(words2)
    if not len(words1) or not len(words2):
        return 0.0
    return len(words1.intersection(words2))/(np.log10(len(words1)) + np.log10(len(words2)))

def your_super_words_similarity(words1, words2):
    # Your code
    sent = [words1, words2]
    tokens, token_embs, subtokens, subtoken_embs, sent_max_embs, sent_mean_embs, bert_pooler_outputs = model(sent)
    similarity = scipy.spatial.distance.cosine(bert_pooler_outputs[0], bert_pooler_outputs[1])

    return similarity

def gen_text_rank_summary(text, calc_similarity=unique_words_similarity, summary_part=0.1, lower=True, morph=None):
    '''
    Составление summary с помощью TextRank
    '''
    # Разбиваем текст на предложения
    sentences = [sentence.text for sentence in razdel.sentenize(text)]
    n_sentences = len(sentences)

    # Токенизируем предложения
    sentences_words = [[token.text.lower() if lower else token.text for token in razdel.tokenize(sentence)] for sentence in sentences]

    # При необходимости лемматизируем слова
    if morph is not None:
        sentences_words = [[morph.parse(word)[0].normal_form for word in words] for words in sentences_words]

    # Для каждой пары предложений считаем близость
    pairs = combinations(range(n_sentences), 2)
    scores = [(i, j, calc_similarity(sentences_words[i], sentences_words[j])) for i, j in pairs]

    # Строим граф с рёбрами, равными близости между предложениями
    g = nx.Graph()
    g.add_weighted_edges_from(scores)

    # Считаем PageRank
    pr = nx.pagerank(g)
    result = [(i, pr[i], s) for i, s in enumerate(sentences) if i in pr]
    result.sort(key=lambda x: x[1], reverse=True)

    # Выбираем топ предложений
    n_summary_sentences = max(int(n_sentences * summary_part), 1)
    result = result[:n_summary_sentences]

    # Восстанавливаем оригинальный их порядок
    result.sort(key=lambda x: x[0])

    # Восстанавливаем текст выжимки
    predicted_summary = " ".join([sentence for i, proba, sentence in result])
    predicted_summary = predicted_summary.lower() if lower else predicted_summary
    return predicted_summary

def calc_text_rank_score(records, calc_similarity=unique_words_similarity, summary_part=0.1, lower=True, nrows=1000, morph=None):
    references = []
    predictions = []

    for text, summary in records[['text', 'summary']].values[:nrows]:
        summary = summary if not lower else summary.lower()
        references.append(summary)

        predicted_summary = gen_text_rank_summary(text, calc_similarity, summary_part, lower, morph=morph)
        text = text if not lower else text.lower()
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

calc_text_rank_score(test_records, calc_similarity=unique_words_similarity)
calc_text_rank_score(test_records, calc_similarity=your_super_words_similarity)

Count: 1000
Ref: украинская писательница лариса ницой раскритиковала участников и членов жюри нацотбора на «евровидение-2020» за использование русского языка. она заявила, что одна из артисток «пренебрежительно» относится к родному языку и назвала «козлами» артиста андрея данилко (верка сердючка) и продюсера руслана квинту.
Hyp: «перепрыгивает с украинского на русский, даже не замечая этого, — возмутилась она. «спецразработка, чтобы, не дай бог, не заговорили все на украинском», — уверена ницой. в 2019 году украину на «евровидении» должна была представлять певица maruv, но в предложенном контракте ее не устроил пункт о запрете выступать в россии и ряд других жестких требований.
BLEU:  0.27462959007731597
ROUGE:  {'rouge-1': {'f': 0.1598688066977366, 'p': 0.13265208579998486, 'r': 0.2159307810626273}, 'rouge-2': {'f': 0.03648547206749619, 'p': 0.029489569984196617, 'r': 0.05183898329233899}, 'rouge-l': {'f': 0.12713362237801293, 'p': 0.11797839871172253, 'r': 0.19237418967548445}}
Count

## 2 Задание: Extractive RNN (порог: 0.3 BLEU)

Второй метод, который вам предлагается улучшить – поиск предложений для summary с помощью RNN. В рассмотренной методе мы использовали LSTM для генерации sentence embedding. Попробуйте использовать другие архитектуры: CNN, Transformer; или добавьте предобученные модели, как и в первом задании.

P.S. Тут предполагается, что придется изменять много кода в ячееках (например, поменять токенизацию). 

### Модель

Картинка для привлечения внимания:

![img](https://storage.googleapis.com/groundai-web-prod/media%2Fusers%2Fuser_14%2Fproject_398421%2Fimages%2Farchitecture.png)

Статья с оригинальным методом:
https://arxiv.org/pdf/1611.04230.pdf

Список вдохновения: 
- https://towardsdatascience.com/understanding-how-convolutional-neural-network-cnn-perform-text-classification-with-word-d2ee64b9dd0b Пример того, как можно применять CNN в текстовых задачах
- https://arxiv.org/pdf/1808.08745.pdf Очень крутой метод генерации summary без Transformers
- https://towardsdatascience.com/super-easy-way-to-get-sentence-embedding-using-fasttext-in-python-a70f34ac5b7c – простой метод генерации sentence embedding
- https://towardsdatascience.com/fse-2b1ffa791cf9 – Необычный метод генерации sentence embedding
- https://github.com/UKPLab/sentence-transformers – BERT предобученный для sentence embedding

P.S. Выше написанные ссылки нужны только для разогрева вашей фантазии, можно воспользоваться ими, а можно придумать свой.

In [0]:
%%time
import copy
import random

def build_oracle_summary_greedy(text, gold_summary, calc_score, lower=True, max_sentences=30):
    '''
    Жадное построение oracle summary
    '''
    gold_summary = gold_summary.lower() if lower else gold_summary
    # Делим текст на предложения
    sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
    n_sentences = len(sentences)
    oracle_summary_sentences = set()
    score = -1.0
    summaries = []
    for _ in range(min(n_sentences, 10)):
        for i in range(n_sentences):
            if i in oracle_summary_sentences:
                continue
            current_summary_sentences = copy.copy(oracle_summary_sentences)
            # Добавляем какое-то предложения к уже существующему summary
            current_summary_sentences.add(i)
            current_summary = " ".join([sentences[index] for index in sorted(list(current_summary_sentences))])
            # Считаем метрики
            current_score = calc_score(current_summary, gold_summary)
            summaries.append((current_score, current_summary_sentences))
        # Если получилось улучшить метрики с добавлением какого-либо предложения, то пробуем добавить ещё
        # Иначе на этом заканчиваем
        best_summary_score, best_summary_sentences = max(summaries)
        if best_summary_score <= score:
            break
        oracle_summary_sentences = best_summary_sentences
        score = best_summary_score
    oracle_summary = " ".join([sentences[index] for index in sorted(list(oracle_summary_sentences))])
    return oracle_summary, oracle_summary_sentences

def calc_single_score(pred_summary, gold_summary, rouge):
    return rouge.get_scores([pred_summary], [gold_summary], avg=True)['rouge-2']['f']

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 15.3 µs


In [0]:
%%time
from tqdm import tqdm_notebook as tqdm

def calc_oracle_score(records, nrows=1000, lower=True):
    references = []
    predictions = []
    rouge = Rouge()
  
    for text, summary in tqdm(records[['text', 'summary']].values[:nrows]):
        summary = summary if not lower else summary.lower()
        references.append(summary)
        predicted_summary, _ = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge))
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

calc_oracle_score(test_records)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



Count: 1000
Ref: в россии упростили механизм взыскания долгов с дачников. если раньше садоводческие товарищества возвращали их годами и через суд, то теперь это можно сделать всего за месяц и гораздо проще.
Hyp: в россии вступили в силу изменения в гражданский процессуальный кодекс, которые упрощают процедуру взыскания долгов дачников перед их садоводческими товариществами, пишет « российская газета ». раньше садоводческие товарищества должны были обращаться с иском в суд. однако некоторые не платят годами и накопили долг в 50 и более тысяч рублей.
BLEU:  0.532084710070161
ROUGE:  {'rouge-1': {'f': 0.36574656214034473, 'p': 0.4029093480237559, 'r': 0.3614917044410697}, 'rouge-2': {'f': 0.20503529670892753, 'p': 0.2319005332294102, 'r': 0.20086025531982027}, 'rouge-l': {'f': 0.3183738666138893, 'p': 0.37329373278291683, 'r': 0.3339666881497283}}
CPU times: user 3min 19s, sys: 798 ms, total: 3min 20s
Wall time: 3min 20s


## (!)
Если надо, поменяйте код загрузки токенизатора

In [0]:
import os

import youtokentome as yttm

def train_bpe(records, model_path, model_type="bpe", vocab_size=10000, lower=True):
    temp_file_name = "temp.txt"
    with open(temp_file_name, "w") as temp:
        for text, summary in records[['text', 'summary']].values:
            if lower:
                summary = summary.lower()
                text = text.lower()
            if not text or not summary:
                continue
            temp.write(text + "\n")
            temp.write(summary + "\n")
    yttm.BPE.train(data=temp_file_name, vocab_size=vocab_size, model=model_path)

train_bpe(train_records, "BPE_model.bin")

In [0]:
import youtokentome as yttm

bpe_processor = yttm.BPE('BPE_model.bin')
bpe_processor.encode(["октябрь богат на изменения"], output_type=yttm.OutputType.SUBWORD)

[['▁окт', 'ябрь', '▁бога', 'т', '▁на', '▁изменения']]

## (!)
Если надо, поменяйте код словаря

In [0]:
from collections import Counter
from typing import List, Tuple
import os

class Vocabulary:
    def __init__(self, bpe_processor):
        self.index2word = bpe_processor.vocab()
        self.word2index = {w: i for i, w in enumerate(self.index2word)}
        self.word2count = Counter()

    def get_pad(self):
        return self.word2index["<PAD>"]

    def get_sos(self):
        return self.word2index["<SOS>"]

    def get_eos(self):
        return self.word2index["<EOS>"]

    def get_unk(self):
        return self.word2index["<UNK>"]
    
    def has_word(self, word) -> bool:
        return word in self.word2index

    def get_index(self, word):
        if word in self.word2index:
            return self.word2index[word]
        return self.get_unk()

    def get_word(self, index):
        return self.index2word[index]

    def size(self):
        return len(self.index2word)

    def is_empty(self):
        empty_size = 4
        return self.size() <= empty_size

    def reset(self):
        self.word2count = Counter()
        self.index2word = ["<pad>", "<sos>", "<eos>", "<unk>"]
        self.word2index = {word: index for index, word in enumerate(self.index2word)}

In [0]:
vocabulary = Vocabulary(bpe_processor)
vocabulary.size()

10000

In [0]:
%%time
from rouge import Rouge
import razdel

def add_oracle_summary_to_records(records, max_sentences=30, lower=True, nrows=1000):
    rouge = Rouge()
    sentences_ = []
    oracle_sentences_ = []
    oracle_summary_ = []
    records = records.iloc[:nrows].copy()

    for text, summary in tqdm(records[['text', 'summary']].values):
        summary = summary.lower() if lower else summary
        sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
        oracle_summary, sentences_indicies = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge),
                                                                         lower=lower, max_sentences=max_sentences)
        sentences_ += [sentences]
        oracle_sentences_ += [list(sentences_indicies)]
        oracle_summary_ += [oracle_summary]
    records['sentences'] = sentences_
    records['oracle_sentences'] = oracle_sentences_
    records['oracle_summary'] = oracle_summary_
    return records

ext_train_records = add_oracle_summary_to_records(train_records, nrows=12288) ### 
ext_val_records = add_oracle_summary_to_records(val_records, nrows=768)###
ext_test_records = add_oracle_summary_to_records(test_records, nrows=768)###

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()



CPU times: user 37min 20s, sys: 7.8 s, total: 37min 27s
Wall time: 37min 24s


## (!)
Если надо, поменяйте код генератора датасета и батчевалки

In [0]:
import random
import math
import razdel
import torch
import numpy as np
from rouge import Rouge


from torch.utils import data


class ExtDataset(data.Dataset):
    def __init__(self, records, vocabulary, bpe_processor, lower=True, max_sentences=30, max_sentence_length=50, device=torch.device('cpu')):
        self.records = records
        self.num_samples = records.shape[0]
        self.bpe_processor = bpe_processor
        self.lower = lower
        self.rouge = Rouge()
        self.vocabulary = vocabulary
        self.max_sentences = max_sentences
        self.max_sentence_length = max_sentence_length
        self.device = device
        
    def __len__(self):
        return self.records.shape[0]

    def __getitem__(self, idx):
        cur_record = self.records.iloc[idx]
        inputs = list(map(lambda x: x[:self.max_sentence_length], self.bpe_processor.encode(cur_record['sentences'], output_type=yttm.OutputType.ID)))
        outputs = [int(i in cur_record['oracle_sentences']) for i in range(len(cur_record['sentences']))]
        return {'inputs': inputs, 'outputs': outputs}

In [0]:
train_dataset = ExtDataset(ext_train_records, vocabulary, bpe_processor)

In [0]:
print(train_dataset[0])

{'inputs': [[4175, 7116, 4203, 2905, 281, 1318, 1429, 338, 1227, 6553, 1567, 6262, 3064, 4445, 856, 2952, 1276, 340, 4940, 1172, 1058, 5388, 3668, 5640, 275, 4111, 558, 7566, 9289, 297, 5370, 309, 1380, 1292, 276, 452, 39, 6008, 2835, 28, 288, 390, 286, 5910, 714, 286, 3288, 4207, 784, 1517], [9287, 883, 344, 387, 368, 919, 1378, 5414, 4741, 3714, 286, 999, 1736], [705, 318, 4305, 495, 1455, 6593, 9772, 29], [1712, 459, 3688, 338, 2084, 1900, 4024, 1156, 399, 344, 1201, 1619, 2580, 3655, 300, 287, 2095, 486, 976, 696], [484, 4846, 714, 2598, 317, 7477, 702, 276, 1844, 540, 6874, 286, 5910, 3288, 2707, 368, 555, 4727, 495, 8077, 425, 3586, 25, 3830, 380, 971, 1975], [484, 562, 4846, 7592, 3489, 2985, 540, 481, 316, 6997, 286, 1698, 308, 353, 4332, 2346, 918, 2643, 2487, 2324, 8385, 29], [5825, 276, 3760, 918, 2143, 2933, 25, 5225, 4525, 2819, 315, 1134, 2610, 9, 519, 399, 7388, 283, 355, 5457, 286, 1547, 304, 359, 493, 36, 7422, 326, 1410, 9420, 4950, 276, 2432, 4812, 7917, 7590, 313, 3

In [0]:
# Это батчевалка
def collate_fn(records):
    max_length = max(len(sentence) for record in records for sentence in record['inputs'])
    max_sentences = max(len(record['outputs']) for record in records)

    new_inputs = torch.zeros((len(records), max_sentences, max_length))
    new_outputs = torch.zeros((len(records), max_sentences))
    for i, record in enumerate(records):
        for j, sentence in enumerate(record['inputs']):
            new_inputs[i, j, :len(sentence)] += np.array(sentence)
        new_outputs[i, :len(record['outputs'])] += np.array(record['outputs'])
    return {'features': new_inputs.type(torch.LongTensor), 'targets': new_outputs}

In [0]:
import transformers
model_name = 'DeepPavlov/rubert-base-cased'
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [0]:
!pip install sentence-transformers

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=d26b3df89b79617f30ccb2e72e6cf57eb70640463c9a1ce5493745be22e143dd
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
Successfully built sentence-transformers


In [0]:
from sentence_transformers import SentenceTransformer, models

embedding_model_word = models.Transformer('DeepPavlov/rubert-base-cased')

pooling_model = models.Pooling(embedding_model_word.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

transformers_model_sentence = SentenceTransformer(modules=[embedding_model_word, pooling_model])

In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack


class SentenceEncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, n_layers=3, dropout=0.3, bidirectional=True):
        super(SentenceEncoderRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.embedding_dim = embedding_dim
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.embedding_layer = nn.Embedding(input_size, embedding_dim)
        self.rnn_layer = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, inputs, hidden=None):
        embedded = self.embedding_layer(inputs)
        outputs, _ = self.rnn_layer(embedded, hidden)
        sentences_embeddings = torch.mean(outputs, 1)
        return sentences_embeddings


class YourSentenceEncoder(nn.Module):
    # Место для вашего Sentence Encoder-а. Разрешается использовать любые методы, которые вам нравятся.
    def __init__(self, input_size, embedding_dim, hidden_size, n_layers=3, dropout=0.3, bidirectional=True):
        super(YourSentenceEncoder, self).__init__()

    def forward(self, inputs, hidden=None):
      sentences = []
      for i in inputs:
          sentences.append(bert_tokenizer.decode(i, skip_special_tokens=True))
      
      return torch.tensor(transformers_model_sentence.encode(sentences)).to(device)


class SentenceTaggerRNN(nn.Module):
    def __init__(self,
                 vocabulary_size,
                 sentence_encoder=SentenceEncoderRNN,
                 token_embedding_dim=256,
                 sentence_encoder_hidden_size=256,
                 hidden_size=256,
                 bidirectional=True,
                 sentence_encoder_n_layers=2,
                 sentence_encoder_dropout=0.3,
                 sentence_encoder_bidirectional=True,
                 n_layers=1,
                 dropout=0.3):
        super(SentenceTaggerRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.sentence_encoder = sentence_encoder(vocabulary_size, token_embedding_dim,
                                                   sentence_encoder_hidden_size, sentence_encoder_n_layers, 
                                                   sentence_encoder_dropout, sentence_encoder_bidirectional) ####
        self.rnn_layer = nn.LSTM(sentence_encoder_hidden_size, hidden_size, n_layers, dropout=dropout,
                           bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)
        self.content_linear_layer = nn.Linear(hidden_size * 2, 1)
        self.document_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.salience_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.tanh_layer = nn.Tanh()

    def forward(self, inputs, hidden=None):
        # print('INPUTS=', inputs)
        batch_size = inputs.size(0)
        sentences_count = inputs.size(1)
        tokens_count = inputs.size(2)
        inputs = inputs.reshape(-1, tokens_count)
        embedded_sentences = self.sentence_encoder(inputs)
        embedded_sentences = embedded_sentences.reshape(batch_size, sentences_count, -1)
        outputs, _ = self.rnn_layer(embedded_sentences, hidden)
        outputs = self.dropout_layer(outputs)
        document_embedding = self.tanh_layer(self.document_linear_layer(torch.mean(outputs, 1)))
        content = self.content_linear_layer(outputs).squeeze(2)
        salience = torch.bmm(outputs, self.salience_linear_layer(document_embedding).unsqueeze(2)).squeeze(2)
        return content + salience

model = SentenceTaggerRNN(vocabulary.size(), sentence_encoder=YourSentenceEncoder, token_embedding_dim=768, sentence_encoder_hidden_size=768)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1



### Обучение

In [0]:
%%time
import catalyst
from catalyst.dl.runner import SupervisedRunner

device = torch.device('cuda')

loaders = {
    'train': data.DataLoader(ExtDataset(ext_train_records, vocabulary, bpe_processor=bpe_processor), batch_size=128, collate_fn=collate_fn),
    'valid': data.DataLoader(ExtDataset(ext_val_records, vocabulary, bpe_processor=bpe_processor), batch_size=128, collate_fn=collate_fn),
    'test': data.DataLoader(ExtDataset(ext_test_records, vocabulary, bpe_processor=bpe_processor), batch_size=128, collate_fn=collate_fn),
}

lr = 1e-3
num_epochs = 10

optimizer  = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()
runner = SupervisedRunner()
runner.train(
    model=model,
    optimizer=optimizer,
    loaders=loaders,
    logdir='./logs',
    num_epochs=num_epochs,
    criterion=criterion,
    verbose=True,
    load_best_on_end=True 
)



1/10 * Epoch (train):   0% 0/96 [00:00<?, ?it/s]
1/10 * Epoch (train):   0% 0/96 [00:40<?, ?it/s, loss=0.606]
1/10 * Epoch (train):   1% 1/96 [00:40<1:03:54, 40.36s/it, loss=0.606]
1/10 * Epoch (train):   1% 1/96 [01:20<1:03:54, 40.36s/it, loss=0.369]
1/10 * Epoch (train):   2% 2/96 [01:20<1:03:13, 40.36s/it, loss=0.369]
1/10 * Epoch (train):   2% 2/96 [02:00<1:03:13, 40.36s/it, loss=0.315]
1/10 * Epoch (train):   3% 3/96 [02:00<1:02:16, 40.18s/it, loss=0.315]
1/10 * Epoch (train):   3% 3/96 [02:40<1:02:16, 40.18s/it, loss=0.318]
1/10 * Epoch (train):   4% 4/96 [02:40<1:01:24, 40.05s/it, loss=0.318]
1/10 * Epoch (train):   4% 4/96 [03:20<1:01:24, 40.05s/it, loss=0.260]
1/10 * Epoch (train):   5% 5/96 [03:20<1:00:46, 40.07s/it, loss=0.260]
1/10 * Epoch (train):   5% 5/96 [04:00<1:00:46, 40.07s/it, loss=0.274]
1/10 * Epoch (train):   6% 6/96 [04:00<1:00:04, 40.05s/it, loss=0.274]
1/10 * Epoch (train):   6% 6/96 [04:40<1:00:04, 40.05s/it, loss=0.290]
1/10 * Epoch (train):   7% 7/96 [04:4

INFO:metrics_logger:
1/10 * Epoch 1 (_base): lr=0.0010 | momentum=0.9000
1/10 * Epoch 1 (train): loss=0.2449
1/10 * Epoch 1 (valid): loss=0.2361
1/10 * Epoch 1 (test): loss=0.2471



2/10 * Epoch (train):   0% 0/96 [00:00<?, ?it/s]
2/10 * Epoch (train):   0% 0/96 [00:39<?, ?it/s, loss=0.230]
2/10 * Epoch (train):   1% 1/96 [00:39<1:02:57, 39.77s/it, loss=0.230]
2/10 * Epoch (train):   1% 1/96 [01:20<1:02:57, 39.77s/it, loss=0.232]
2/10 * Epoch (train):   2% 2/96 [01:20<1:02:34, 39.94s/it, loss=0.232]
2/10 * Epoch (train):   2% 2/96 [02:00<1:02:34, 39.94s/it, loss=0.228]
2/10 * Epoch (train):   3% 3/96 [02:00<1:01:56, 39.96s/it, loss=0.228]
2/10 * Epoch (train):   3% 3/96 [02:39<1:01:56, 39.96s/it, loss=0.229]
2/10 * Epoch (train):   4% 4/96 [02:40<1:01:14, 39.94s/it, loss=0.229]
2/10 * Epoch (train):   4% 4/96 [03:20<1:01:14, 39.94s/it, loss=0.235]
2/10 * Epoch (train):   5% 5/96 [03:20<1:00:48, 40.09s/it, loss=0.235]
2/10 * Epoch (train):   5% 5/96 [04:00<1:00:48, 40.09s/it, loss=0.239]
2/10 * Epoch (train):   6% 6/96 [04:00<1:00:13, 40.15s/it, loss=0.239]
2/10 * Epoch (train):   6% 6/96 [04:41<1:00:13, 40.15s/it, loss=0.241]
2/10 * Epoch (train):   7% 7/96 [04:4

INFO:metrics_logger:
2/10 * Epoch 2 (_base): lr=0.0010 | momentum=0.9000
2/10 * Epoch 2 (train): loss=0.2319
2/10 * Epoch 2 (valid): loss=0.2338
2/10 * Epoch 2 (test): loss=0.2458



3/10 * Epoch (train):   0% 0/96 [00:00<?, ?it/s]
3/10 * Epoch (train):   0% 0/96 [00:39<?, ?it/s, loss=0.226]
3/10 * Epoch (train):   1% 1/96 [00:39<1:02:43, 39.62s/it, loss=0.226]
3/10 * Epoch (train):   1% 1/96 [01:19<1:02:43, 39.62s/it, loss=0.228]
3/10 * Epoch (train):   2% 2/96 [01:19<1:02:20, 39.79s/it, loss=0.228]
3/10 * Epoch (train):   2% 2/96 [01:59<1:02:20, 39.79s/it, loss=0.227]
3/10 * Epoch (train):   3% 3/96 [01:59<1:01:36, 39.75s/it, loss=0.227]
3/10 * Epoch (train):   3% 3/96 [02:39<1:01:36, 39.75s/it, loss=0.226]
3/10 * Epoch (train):   4% 4/96 [02:39<1:00:53, 39.72s/it, loss=0.226]
3/10 * Epoch (train):   4% 4/96 [03:19<1:00:53, 39.72s/it, loss=0.236]
3/10 * Epoch (train):   5% 5/96 [03:19<1:00:26, 39.85s/it, loss=0.236]
3/10 * Epoch (train):   5% 5/96 [03:59<1:00:26, 39.85s/it, loss=0.236]
3/10 * Epoch (train):   6% 6/96 [03:59<59:51, 39.91s/it, loss=0.236]  
3/10 * Epoch (train):   6% 6/96 [04:39<59:51, 39.91s/it, loss=0.239]
3/10 * Epoch (train):   7% 7/96 [04:39<

...


6/10 * Epoch (train):   0% 0/96 [00:00<?, ?it/s]
6/10 * Epoch (train):   0% 0/96 [00:39<?, ?it/s, loss=0.225]
6/10 * Epoch (train):   1% 1/96 [00:39<1:02:21, 39.38s/it, loss=0.225]
6/10 * Epoch (train):   1% 1/96 [01:19<1:02:21, 39.38s/it, loss=0.226]
6/10 * Epoch (train):   2% 2/96 [01:19<1:02:02, 39.61s/it, loss=0.226]
6/10 * Epoch (train):   2% 2/96 [01:59<1:02:02, 39.61s/it, loss=0.224]
6/10 * Epoch (train):   3% 3/96 [01:59<1:01:23, 39.61s/it, loss=0.224]
6/10 * Epoch (train):   3% 3/96 [02:38<1:01:23, 39.61s/it, loss=0.224]
6/10 * Epoch (train):   4% 4/96 [02:38<1:00:45, 39.62s/it, loss=0.224]
6/10 * Epoch (train):   4% 4/96 [03:19<1:00:45, 39.62s/it, loss=0.233]
6/10 * Epoch (train):   5% 5/96 [03:19<1:00:26, 39.85s/it, loss=0.233]
6/10 * Epoch (train):   5% 5/96 [03:59<1:00:26, 39.85s/it, loss=0.234]
6/10 * Epoch (train):   6% 6/96 [03:59<59:49, 39.88s/it, loss=0.234]  
6/10 * Epoch (train):   6% 6/96 [04:39<59:49, 39.88s/it, loss=0.237]
6/10 * Epoch (train):   7% 7/96 [04:39<

INFO:metrics_logger:
6/10 * Epoch 6 (_base): lr=0.0010 | momentum=0.9000
6/10 * Epoch 6 (train): loss=0.2283
6/10 * Epoch 6 (valid): loss=0.2316
6/10 * Epoch 6 (test): loss=0.2438



7/10 * Epoch (train):   0% 0/96 [00:00<?, ?it/s]
7/10 * Epoch (train):   0% 0/96 [00:39<?, ?it/s, loss=0.223]
7/10 * Epoch (train):   1% 1/96 [00:39<1:03:03, 39.83s/it, loss=0.223]
7/10 * Epoch (train):   1% 1/96 [01:20<1:03:03, 39.83s/it, loss=0.227]
7/10 * Epoch (train):   2% 2/96 [01:20<1:02:42, 40.02s/it, loss=0.227]
7/10 * Epoch (train):   2% 2/96 [02:00<1:02:42, 40.02s/it, loss=0.224]
7/10 * Epoch (train):   3% 3/96 [02:00<1:02:01, 40.01s/it, loss=0.224]
7/10 * Epoch (train):   3% 3/96 [02:40<1:02:01, 40.01s/it, loss=0.222]
7/10 * Epoch (train):   4% 4/96 [02:40<1:01:26, 40.07s/it, loss=0.222]
7/10 * Epoch (train):   4% 4/96 [03:21<1:01:26, 40.07s/it, loss=0.231]
7/10 * Epoch (train):   5% 5/96 [03:21<1:00:59, 40.22s/it, loss=0.231]
7/10 * Epoch (train):   5% 5/96 [04:01<1:00:59, 40.22s/it, loss=0.234]
7/10 * Epoch (train):   6% 6/96 [04:01<1:00:28, 40.31s/it, loss=0.234]
7/10 * Epoch (train):   6% 6/96 [04:41<1:00:28, 40.31s/it, loss=0.237]
7/10 * Epoch (train):   7% 7/96 [04:4

INFO:metrics_logger:
7/10 * Epoch 7 (_base): lr=0.0010 | momentum=0.9000
7/10 * Epoch 7 (train): loss=0.2276
7/10 * Epoch 7 (valid): loss=0.2320
7/10 * Epoch 7 (test): loss=0.2442



8/10 * Epoch (train):   0% 0/96 [00:00<?, ?it/s]
8/10 * Epoch (train):   0% 0/96 [00:40<?, ?it/s, loss=0.223]
8/10 * Epoch (train):   1% 1/96 [00:40<1:03:25, 40.05s/it, loss=0.223]
8/10 * Epoch (train):   1% 1/96 [01:20<1:03:25, 40.05s/it, loss=0.226]
8/10 * Epoch (train):   2% 2/96 [01:20<1:02:46, 40.07s/it, loss=0.226]
8/10 * Epoch (train):   2% 2/96 [01:59<1:02:46, 40.07s/it, loss=0.222]
8/10 * Epoch (train):   3% 3/96 [01:59<1:01:53, 39.93s/it, loss=0.222]
8/10 * Epoch (train):   3% 3/96 [02:39<1:01:53, 39.93s/it, loss=0.221]
8/10 * Epoch (train):   4% 4/96 [02:39<1:01:19, 40.00s/it, loss=0.221]
8/10 * Epoch (train):   4% 4/96 [03:20<1:01:19, 40.00s/it, loss=0.230]
8/10 * Epoch (train):   5% 5/96 [03:20<1:00:53, 40.14s/it, loss=0.230]
8/10 * Epoch (train):   5% 5/96 [04:00<1:00:53, 40.14s/it, loss=0.232]
8/10 * Epoch (train):   6% 6/96 [04:00<1:00:17, 40.20s/it, loss=0.232]
8/10 * Epoch (train):   6% 6/96 [04:41<1:00:17, 40.20s/it, loss=0.237]
8/10 * Epoch (train):   7% 7/96 [04:4

In [0]:
%%time
device = torch.device("cuda")

references = []
predictions = []
model.eval()
for i, item in tqdm(enumerate(data.DataLoader(ExtDataset(ext_test_records, vocabulary, bpe_processor=bpe_processor), batch_size=1, collate_fn=collate_fn)), total=ext_test_records.shape[0]):
    logits = model(item["features"].to(device))[0] # Прямой проход
    record = ext_test_records.iloc[i]
    predicted_summary = []
    for i, logit in enumerate(logits):
        if logit > 0.0:
            predicted_summary.append(record['sentences'][i])
    if not predicted_summary:
        predicted_summary.append(record['sentences'][torch.max(logits, dim=0)[1].item()])
    predicted_summary = " ".join(predicted_summary)
    references.append(record['summary'].lower())
    predictions.append(predicted_summary)

calc_scores(references, predictions)